Dependencies

In [ ]:
%cd /home/ubuntu/invoice_parser

In [1]:
import pytesseract
from pytesseract import Output

ModuleNotFoundError: No module named 'pytesseract'

In [3]:
!pip3 install pytesseract

/bin/bash: /home/ubuntu/.local/bin/pip3: /home/ubuntu/environments/invoice_test/bin/python3: bad interpreter: No such file or directory


In [ ]:
!pip install pdf2image
!git clone https://github.com/PaddlePaddle/PaddleOCR

In [ ]:
!sudo apt-get install poppler-utils
!pip install paddleocr
!python -m pip install paddlepaddle-gpu==2.0.0 -i https://mirror.baidu.com/pypi/simple
!pip install paddlepaddle
!pip install protobuf==3.19.0

image preprocessing

In [ ]:
from paddleocr import PaddleOCR,draw_ocr
from matplotlib import pyplot as plt
import cv2
import os
from google.colab.patches import cv2_imshow
from pdf2image import convert_from_path
from PIL import Image 
import PIL 
import os

In [ ]:
# Directory
directory = "converted_pdf"
  
# Parent Directory path
parent_dir = "/content"
path = os.path.join(parent_dir, directory)
os.mkdir(path)
new_dir = "/content/converted_pdf"

dir = '/content/invoice'
for index,filename in enumerate(os.listdir(dir)):
    file_dir = str(dir)+"/"+filename
    # Store Pdf with convert_from_path function\
    images = convert_from_path(file_dir) #dpi = 100
    for i in range(len(images)):
        # Save pages as images in the pdf
        images[i].save(str(new_dir)+'/'+str(filename)+"_image"+str(i)+'.png', 'PNG')

In [ ]:
# Renaming the file created
temp_dir = '/content/converted_pdf'
for i in os.listdir(temp_dir):
    a = i.replace('.pdf','')
    new_filename = str(temp_dir)+'/'+a
    old_filename = str(temp_dir)+'/'+i
    os.rename(old_filename,new_filename)

check the image orientation

In [ ]:
temp_dir = '/content/converted_pdf'

In [ ]:
import numpy as np
from PIL import Image, ImageEnhance

In [ ]:
for i in os.listdir(temp_dir):
  t_dir = temp_dir+"/"+i
  image = cv2.imread(t_dir)
  # Convert the image to grayscale
  gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
  # Use the Canny edge detection algorithm to detect edges in the image
  edges = cv2.Canny(gray, 50, 150)
  # Calculate the histograms of the edges in the x and y direction
  histogram_x = cv2.reduce(edges, 0, cv2.REDUCE_AVG).reshape(-1)
  histogram_y = cv2.reduce(edges, 1, cv2.REDUCE_AVG).reshape(-1)
  # Calculate the standard deviation of the histograms
  std_x = np.std(histogram_x)
  std_y = np.std(histogram_y)
  # If the standard deviation in the x direction is greater than the standard deviation in the y direction,
  # the image is likely to be rotated
  if std_x > std_y:
      print("The image is rotated")
      im = Image.open(t_dir)
      ima = im.rotate(270, expand=True)
      #im.show()
      ima.save(t_dir)
  else:
      print("The image is straight") 

ocr multiple images

In [ ]:
# Load the ocr model
ocr = PaddleOCR(lang='en')

In [ ]:
# Directory
directory = "OCR_out"
  
# Parent Directory path
parent_dir = "/content"
path = os.path.join(parent_dir, directory)
os.mkdir(path)
new_dir = "/content/OCR_out"

In [ ]:
dir = '/content/converted_pdf'
#f = open(new_file_dir+'/'+"ocr_out.txt", "a")
for file_name in os.listdir(dir):
  #count = 0
  img_path = dir+'/'+file_name
  results = ocr.ocr(img_path)
  # print(results)
  #f = open(new_file_dir+'/'+"ocr_out.txt", "a")
  img = cv2.imread(img_path)
  scores = []
  boxes = []
  texts = []
  for i in range(len(results[0])):
    scores.append(results[0][i][1][1])
    boxes.append(results[0][i][0])
    texts.append(results[0][i][1][0])
  font_path = '/content/PaddleOCR/doc/fonts/latin.ttf'
  annotated = draw_ocr(img, boxes, texts, scores, font_path = font_path)
  cv2_imshow(annotated)
  #count+=1
  cv2.imwrite(new_dir+'/'+str(file_name)+'.png',annotated)

In [ ]:
# Renaming the file created
temp_dir = '/content/OCR_out'
for i in os.listdir(temp_dir):
    a = i.replace('.png.png','.png')
    new_filename = str(temp_dir)+'/'+a
    old_filename = str(temp_dir)+'/'+i
    os.rename(old_filename,new_filename)

#single pdf

In [ ]:
pdf_path = '/content/Payment Challan-Sample.pdf'

In [ ]:
images = convert_from_path(pdf_path)

In [ ]:
for i in range(len(images)):
      # Save pages as images in the pdf
      images[i].save('images.png', 'PNG')

In [ ]:
img_path = '/content/images.png'

In [ ]:
img = Image.open(img_path)

In [ ]:
Rotated_image = img.rotate(270)

In [ ]:
Rotated_image.show()

ocr

In [ ]:
# Load the ocr model
ocr = PaddleOCR(lang='en')

In [ ]:
results = ocr.ocr(img_path)

In [ ]:
results

In [ ]:
img = cv2.imread(img_path)
scores = []
boxes = []
texts = []
for i in range(len(results[0])):
  scores.append(results[0][i][1][1])
  boxes.append(results[0][i][0])
  texts.append(results[0][i][1][0])
font_path = '/content/PaddleOCR/doc/fonts/latin.ttf'
annotated = draw_ocr(img, boxes, texts, scores, font_path = font_path)
cv2_imshow(annotated)
cv2.imwrite('annotated.png',annotated)

# QR Code detection and decoding

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
#clone YOLOv5 and 
!git clone https://github.com/ultralytics/yolov5  # clone repo
%cd yolov5
%pip install -qr requirements.txt # install dependencies
%pip install -q roboflow

import torch
import os
from IPython.display import Image, clear_output  # to display images

print(f"Setup complete. Using torch {torch.__version__} ({torch.cuda.get_device_properties(0).name if torch.cuda.is_available() else 'CPU'})")

In [ ]:
!python detect.py --weights /content/drive/MyDrive/qr_pt/best.pt --img 416 --conf 0.4 --source /content/converted_pdf --save-crop 

In [ ]:
!pip install pyzbar

In [ ]:
!apt install libzbar0

In [ ]:
from pyzbar.pyzbar import decode

In [ ]:
dir = '/content/yolov5/runs/detect/exp/crops/qr_code'
for index,filename in enumerate(os.listdir(dir)): 
    #f = open(new_file_dir+'/'+"qr_code.txt", "a")
    img = cv2.imread(str(dir)+"/"+filename)
    #cv2_imshow(img)
    #detector = cv2.QRCodeDetector()
    decoded_qr = decode(img)
    #print(decoded_qr[0][0])
    #print(filename)
    #print(decoded_qr)
   # if decoded_qr == []:
    #   print("yes")
    # else:
    #   print("No")
    #print("Data:",decoded_qr[0][0])
    #print("Bounding Box:",decoded_qr[0][2])
    # if decoded_qr == []:
    #   f.write(str(filename))
    #   f.write(' ')
    #   f.write('There is no barcode')
    # else:
    #   f.write(str(filename))
    #   f.write(' ')
    #   f.write('Barcode detected')
    #   f.write(' ')
    #   f.write("{Data}:"+str(decoded_qr[0][0]))
    # f.write('\n')
    # f.close()
    #f.write("{Data}:"+str(decoded_qr[0][0]))
    #f.write("{bbox}:"+str(decoded_qr[0][2]))
    # f.write("{straight_qrcode}:"+str(straight_qrcode))
    if decoded_qr == []:
      pass
    else:
      d = decoded_qr[0][2]
      image = cv2.rectangle(img, (d.left, d.top),
                        (d.left + d.width, d.top + d.height), (255, 0, 0), 2)
      #cv2.imwrite(new_dir+'/'+filename+'.png',image)
      cv2_imshow(image)
      print("Decoded link",decoded_qr[0][0])
      print("Bounding box",decoded_qr[0][3])